In [88]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import time
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [49]:
# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

In [50]:
train_images = train_images.reshape((60000,28,28,1))
train_images = train_images/255.0

test_images = test_images.reshape((10000,28,28,1))
test_images = test_images/255.0

Defining our model

In [51]:
# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name='conv2d_1'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', name='conv2d_2'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', name='conv2d_3'),
    tf.keras.layers.Flatten(name='flatten'),
    tf.keras.layers.Dense(10, activation='softmax', name='output')
])

In [52]:
# compile the model
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [53]:
# Train the model and display the activations after each epoch
start_time = time.time()
model.fit(train_images, train_labels, epochs=1, validation_data=(test_images, test_labels))
end_time = time.time()

1875/1875 [==============================] - 69s 37ms/step - loss: 0.1062 - accuracy: 0.9680 - val_loss: 0.0535 - val_accuracy: 0.9824


In [54]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)
print('Time elapsed: ', end_time - start_time)

313/313 [==============================] - 4s 11ms/step - loss: 0.0535 - accuracy: 0.9824
Test accuracy: 0.9824000000953674
Time elapsed:  69.2872998714447


In [55]:
model.save('model_init.h5')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [56]:
from sklearn.model_selection import train_test_split

# Load Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.fashion_mnist.load_data()

# Split the training data into a 50:50 train-test split
train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.9, random_state=42)

In [57]:
train_images = train_images.reshape((6000,28,28,1))
train_images = train_images/255.0

test_images = test_images.reshape((54000,28,28,1))
test_images = test_images/255.0

In [148]:
# Train the model and display the activations after each epoch
start_time = time.time()
model.fit(train_images, train_labels, epochs=1, validation_data=(test_images, test_labels))
end_time = time.time()

188/188 [==============================] - 24s 128ms/step - loss: 0.6810 - accuracy: 0.7627 - val_loss: 0.4485 - val_accuracy: 0.8378


In [149]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)
print('Time elapsed: ', end_time - start_time)

1688/1688 [==============================] - 19s 11ms/step - loss: 0.4485 - accuracy: 0.8378
Test accuracy: 0.8378333449363708
Time elapsed:  24.062577962875366


In [77]:
#reusing the saved model
import tensorflow as tf
from tensorflow import keras

loaded_model = keras.models.load_model('model_init.h5')


In [78]:
for l in loaded_model.layers:
    print(l)
    

In [74]:
# from tensorflow.keras.layers import Dense,Flatten
# from tensorflow.keras import Model

In [79]:
#making changes in the layers of the pre-saved model
for i in range(2):
    loaded_model.pop()

#here we are using the same lower layer layers of the initial model but adding a new top fully connected layers
#so as to adapt with the new target domain

# x = loaded_model.layers[-3].output 
# x = Flatten()(x)
# x = Dense(64,activation='relu',name='dense_1')
# predictions = Dense(10, activation = "softmax")(x)
# final_model = Model(inputs = loaded_model.input, outputs = predictions)

In [81]:
# #adding new flatten and dense layers to help the model adapt the target domain
loaded_model.add(tf.keras.layers.Flatten(name='flatten'))
loaded_model.add(tf.keras.layers.Dense(64, activation='relu', name='dense_1'))
loaded_model.add(tf.keras.layers.Dense(10, activation='softmax', name='output'))
# #loaded_model.add(tf.keras.layers.Dense(128, activation='relu', name='dense_2'))


In [82]:
for l in loaded_model.layers:
    print(l)

In [84]:
#here we freeze the convolutional layers so that its weights and biases are not affected
#we only want to have the top fully connected adjusted as per the target
for layer in loaded_model.layers[:3]:
    layer.trainable = False

In [85]:
# compile the model
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
loaded_model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [86]:

# Train the model and display the activations after each epoch
start_time = time.time()
loaded_model.fit(train_images, train_labels, epochs=1, validation_data=(test_images, test_labels))
end_time = time.time()


188/188 [==============================] - 22s 117ms/step - loss: 0.5878 - accuracy: 0.7958 - val_loss: 0.4564 - val_accuracy: 0.8387


In [87]:
test_loss, test_acc = loaded_model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)
print('Time elapsed: ', end_time - start_time)

1688/1688 [==============================] - 20s 12ms/step - loss: 0.4564 - accuracy: 0.8387
Test accuracy: 0.8386852145195007
Time elapsed:  22.023769855499268
